In [1]:
from nltk.corpus import stopwords
from string import punctuation
from nltk.tokenize import word_tokenize
import pymorphy2
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel

import pandas as pd
import re

In [2]:
# Функция для получения текста вопросов.
def getting_texts():
    # Для ускоренной настройки модели считывается только 1000 первых строчек, но можно указать 'None' для чтения всего файла.
    nRowsRead = 10001
    data = pd.read_csv("quora_question_pairs_rus.csv", delimiter=',', header = 0, nrows = nRowsRead).dropna()
    all_questions_dataset = pd.concat([data['question1'], data['question2']], ignore_index=True)
    all_questions_list = all_questions_dataset.tolist()
    
    return all_questions_list

In [3]:
# Настраиваем лемматизатор, устанавливаем стоп-слова и немного расширяем пунктуацию.
morph_analyze = pymorphy2.MorphAnalyzer()
russian_stopwords = stopwords.words("russian")
punctuation += '«»…—'

# Функция для препроцессинга текста
def preprocessing_text(text):
    lemmas = []
    # Переводим всё в нижний регистр шрифта и делим по пробелам
    pre_tokens = word_tokenize(text.lower())
    # Избавляемся от стоп-слов и пунктуации, лемматизируем
    for one_pre_token in pre_tokens:
        if one_pre_token not in punctuation and one_pre_token not in russian_stopwords:
            one_lemma = morph_analyze.normal_forms(one_pre_token)[0]
            lemmas.append(one_lemma)
    # Превращаем в строки с леммами, разделёнными пробелами
    processed_text = ' '.join(lemmas)
    return processed_text

In [4]:
# Функция, которая создаёт последовательности: исходные вопросы из датасета, они же после препроцессинга, их id.
def sequencing_texts(texts_list):
    # Вспомогательная переменная, здесь будут храниться все тексты, прошедшие предпроцессинг.
    all_questions_processed = []
    # А здесь будут храниться все тексты, которые не прошли предпроцессинг.
    all_questions_raw = texts_list
    for one_question in all_questions_raw:
        # Лемматизируем текст вопроса, избавляемся от стоп-слов и ненужных символов.
        processed_question = preprocessing_text(one_question)
        # Записываем прошедший предобработку вопрос в переменную.
        all_questions_processed.append(processed_question)
    # Делаем массив с id вопросов.
    all_question_ids = [i for i in range(1, len(all_questions_processed) + 1)]
    return all_questions_raw, all_questions_processed, all_question_ids

In [5]:
### MAIN. Делаем первичную обработку данных, представляем их в удобном виде.

all_questions_raw, all_questions_processed, all_question_ids = sequencing_texts(getting_texts())

In [6]:
### Создаём матрицу TF-IDF.
tfidf_vectorizer = TfidfVectorizer()
tfidf = tfidf_vectorizer.fit_transform(all_questions_processed)

In [7]:
def query_preprocessing():
    query = input('Введите строку запроса: ')
    processed_query = preprocessing_text(query)
    query_list = processed_query.split()
    query_matrix = tfidf_vectorizer.transform(query_list)
    return query_matrix

In [8]:
def searching_cosine_similarity(query_matrix):
    cosine_similarities = linear_kernel(query_matrix[0:1], tfidf).flatten()
    related_questions_ids = cosine_similarities.argsort()[:-6:-1]
    related_questions_metrics = cosine_similarities[related_questions_ids]
    res_number = 1
    for i in related_questions_ids:
        metr = related_questions_metrics[res_number-1]
        print('№ ' + str(res_number) + ', метрика = ' + str(metr) + ', вопрос: ' + all_questions_raw[i])
        res_number += 1

In [13]:
searching_cosine_similarity(query_preprocessing())

Введите строку запроса: в чем смысл жизни
№ 1, метрика = 0.8080761171645133, вопрос: в чем смысл жизни
№ 2, метрика = 0.8080761171645133, вопрос: какой смысл этой жизни
№ 3, метрика = 0.8080761171645133, вопрос: в чем смысл жизни
№ 4, метрика = 0.8080761171645133, вопрос: какой смысл этой жизни
№ 5, метрика = 0.8080761171645133, вопрос: в чем смысл жизни
